<a href="https://colab.research.google.com/github/iffasadiq/finalcapstone/blob/main/Dynamic_Pricing_for_Urban_Parking_Lots_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

"""
Dynamic Pricing for Urban Parking Lots - Capstone Project
Summer Analytics 2025

This notebook implements a dynamic pricing engine for urban parking lots based on real-time data.
It includes:
1. Data loading, exploration, and preprocessing.
2. Implementation of three pricing models:
   - Model 1: Baseline Linear Model
   - Model 2: Demand-Based Price Function
   - Model 3: Competitive Pricing Model (Optional)
3. Conceptual integration with Pathway for real-time simulation.
4. Real-time visualizations using Bokeh (demonstrated with historical data).
"""

import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# For Bokeh visualizations
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, DatetimeTickFormatter
from bokeh.palettes import Category10
from bokeh.layouts import column # For arranging multiple plots

# Ensure Bokeh outputs to the notebook
output_notebook()

# --- Global Constants ---
BASE_PRICE = 10.0 # Starting base price for all parking lots
MIN_PRICE_GLOBAL = BASE_PRICE * 0.5 # Minimum allowed price (0.5x base)
MAX_PRICE_GLOBAL = BASE_PRICE * 2.0 # Maximum allowed price (2x base)

# Model 1 Coefficients
ALPHA_MODEL1 = 5.0 # Sensitivity of price to occupancy rate in Model 1

# Model 2 Demand Function Coefficients (tuned for illustrative purposes)
# These coefficients should ideally be determined through regression or domain expertise.
# Positive alpha for OccupancyRate, QueueLength, IsSpecialDay, VehicleTypeWeight
ALPHA_OCCUPANCY = 0.5      
ALPHA_QUEUELENGTH = 0.3    
ALPHA_TRAFFIC = 0.2        
ALPHA_SPECIALDAY = 1.0     
ALPHA_VEHICLETYPE = 0.1    

# Model 2 Price Adjustment Coefficient
LAMBDA_MODEL2 = 0.5 # Sensitivity of price to normalized demand

# Model 3 Competitive Pricing Constants
COMPETITOR_DISTANCE_THRESHOLD_KM = 1.0 # Distance threshold to consider a lot a competitor
ETA_COMPETITIVE = 0.1 # Sensitivity of price to competitive advantage


# --- Milestone 1: Data Understanding, Exploration, and Preprocessing ---

print("--- Milestone 1: Data Understanding, Exploration, and Preprocessing ---")

# 1. Load the dataset
# Using 'dataset.csv' as per the last uploaded file.
DATASET_FILENAME = 'dataset.csv'
print(f"Loading the dataset '{DATASET_FILENAME}'...")
try:
    df = pd.read_csv(DATASET_FILENAME)
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print(f"Error: '{DATASET_FILENAME}' not found. Please ensure the file is in the correct directory (e.g., uploaded to Colab).")
# 2. Display basic information and descriptive statistics
print("\n--- Initial Data Overview ---")
print("\nFirst 5 rows of the dataset:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nDescriptive Statistics:")
print(df.describe())

# 3. Combine date and time columns into a single datetime object and sort
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'])
# Sort by timestamp to simulate chronological data flow for pricing models
df = df.sort_values(by='Timestamp').reset_index(drop=True)
print("\n'Timestamp' column created and dataset sorted by time.")
print(df[['LastUpdatedDate', 'LastUpdatedTime', 'Timestamp']].head())

# 4. Calculate Occupancy Rate
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
# Handle potential division by zero or infinite values if Capacity could be 0
df['OccupancyRate'].replace([np.inf, -np.inf], np.nan, inplace=True)
df['OccupancyRate'].fillna(0, inplace=True) # Fill NaN occupancy rates with 0
df['OccupancyRate'] = df['OccupancyRate'].clip(0, 1) # Ensure occupancy rate is between 0 and 1
print("\n'OccupancyRate' column added and clipped between 0 and 1.")
print(df[['Occupancy', 'Capacity', 'OccupancyRate']].head())

# 5. Encode categorical features
# Define weights for VehicleType. These are assumptions and can be refined.
vehicle_type_weights = {
    'car': 1.0,
    'bike': 0.5, # Bikes take less space, might have lower demand weight
    'truck': 1.5 # Trucks take more space, higher demand weight or special pricing
}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_type_weights)
# Handle potential NaN if a vehicle type not in map appears
df['VehicleTypeWeight'].fillna(1.0, inplace=True) # Default to car weight if unknown type
print("\n'VehicleTypeWeight' column added based on mapping.")
print(df[['VehicleType', 'VehicleTypeWeight']].head())

# Ordinal encoding for TrafficConditionNearby
traffic_mapping = {
    'low': 0,
    'average': 1,
    'high': 2
}
df['TrafficConditionEncoded'] = df['TrafficConditionNearby'].map(traffic_mapping)
# Handle potential NaN if an unknown traffic condition appears
df['TrafficConditionEncoded'].fillna(0, inplace=True) # Default to 'low' traffic if unknown
print("\n'TrafficConditionEncoded' column added based on ordinal mapping.")
print(df[['TrafficConditionNearby', 'TrafficConditionEncoded']].head())

print("\n--- Preprocessing Complete ---")
print("\nDataset after preprocessing (first 5 rows with new columns):")
print(df.head())


# --- Milestone 2: Model 1: Baseline Linear Model Implementation ---

print("\n\n--- Milestone 2: Implementing Model 1: Baseline Linear Model ---")

# Initialize a dictionary to store current prices for each parking lot
# This simulates the state of prices in a real-time system.
current_prices_model1 = {lot_id: BASE_PRICE for lot_id in df['SystemCodeNumber'].unique()}

# Prepare a list to store pricing results for analysis and visualization
model1_pricing_results = []

# Simulate price updates over time, iterating through the sorted dataframe
# In a real-time Pathway system, this logic would apply to incoming data streams.
for index, row in df.iterrows():
    lot_id = row['SystemCodeNumber']
    occupancy_rate = row['OccupancyRate']
    current_time = row['Timestamp']

    # Get the previous price for this lot from the current state
    prev_price = current_prices_model1[lot_id]

    # Calculate the next price based on Model 1 logic: Pricet+1 = Pricet + α · OccupancyRate
    next_price = prev_price + ALPHA_MODEL1 * occupancy_rate

    # Apply global price bounds to ensure smoothness and realism
    next_price = max(MIN_PRICE_GLOBAL, min(MAX_PRICE_GLOBAL, next_price))

    # Update the current price for the lot in the state dictionary
    current_prices_model1[lot_id] = next_price

    # Store the results for this timestamp and lot
    model1_pricing_results.append({
        'Timestamp': current_time,
        'SystemCodeNumber': lot_id,
        'CurrentOccupancyRate': occupancy_rate,
        'Model1_Price': next_price
    })

# Convert results to a DataFrame for easier analysis
model1_df = pd.DataFrame(model1_pricing_results)
print("\nModel 1 simulation complete.")
print("Sample of Model 1 pricing results:")
print(model1_df.head())

# Merge Model 1 prices back to the main DataFrame for comprehensive analysis later
df = df.merge(model1_df[['Timestamp', 'SystemCodeNumber', 'Model1_Price']],
              on=['Timestamp', 'SystemCodeNumber'],
              how='left')
print("\nModel 1 prices merged back to main DataFrame.")
print(df.head())


# --- Milestone 3: Model 2: Demand-Based Price Function Implementation ---

print("\n\n--- Milestone 3: Implementing Model 2: Demand-Based Price Function ---")

# 1. Calculate Raw Demand for all records
# Demand = α1·OccupancyRate + α2·QueueLength + α3·Traffic + α4·IsSpecialDay + α5·VehicleTypeWeight
# Note: We use positive ALPHA_TRAFFIC assuming higher traffic means more people looking for parking.
df['RawDemand'] = (
    ALPHA_OCCUPANCY * df['OccupancyRate'] +
    ALPHA_QUEUELENGTH * df['QueueLength'] +
    ALPHA_TRAFFIC * df['TrafficConditionEncoded'] +
    ALPHA_SPECIALDAY * df['IsSpecialDay'] +
    ALPHA_VEHICLETYPE * df['VehicleTypeWeight']
)
print("\n'RawDemand' calculated for all records based on defined coefficients.")
print(df[['OccupancyRate', 'QueueLength', 'TrafficConditionEncoded', 'IsSpecialDay', 'VehicleTypeWeight', 'RawDemand']].head())

# 2. Normalize Demand
# Normalize based on the full range of 'RawDemand' across the entire dataset.
# This ensures demand is scaled consistently to a [0, 1] range.
min_demand = df['RawDemand'].min()
max_demand = df['RawDemand'].max()

# Handle case where min_demand == max_demand to prevent division by zero
if max_demand == min_demand:
    df['NormalizedDemand'] = 0.0 # All demands are the same, so normalized demand is 0
else:
    df['NormalizedDemand'] = (df['RawDemand'] - min_demand) / (max_demand - min_demand)

print("\n'NormalizedDemand' calculated and scaled to [0, 1].")
print(df[['RawDemand', 'NormalizedDemand']].head())

# 3. Simulate price updates for Model 2 using the normalized demand
# Pricet = BasePrice · (1 + λ · NormalizedDemand)
model2_pricing_results = []

for index, row in df.iterrows():
    normalized_demand = row['NormalizedDemand']
    current_time = row['Timestamp']
    lot_id = row['SystemCodeNumber']

    # Calculate price based on Model 2 logic
    model2_price = BASE_PRICE * (1 + LAMBDA_MODEL2 * normalized_demand)

    # Apply global price bounds
    model2_price = max(MIN_PRICE_GLOBAL, min(MAX_PRICE_GLOBAL, model2_price))

    # Store results
    model2_pricing_results.append({
        'Timestamp': current_time,
        'SystemCodeNumber': lot_id,
        'NormalizedDemand': normalized_demand,
        'Model2_Price': model2_price
    })

# Convert results to a DataFrame
model2_df_results = pd.DataFrame(model2_pricing_results)
print("\nModel 2 simulation complete.")
print("Sample of Model 2 pricing results:")
print(model2_df_results.head())

# Merge Model 2 prices back to the main DataFrame
df = df.merge(model2_df_results[['Timestamp', 'SystemCodeNumber', 'Model2_Price']],
              on=['Timestamp', 'SystemCodeNumber'],
              how='left')
print("\nModel 2 prices merged back to main DataFrame.")
print(df.head())


# --- Milestone 4: Model 3 (Optional): Competitive Pricing Model Implementation ---

print("\n\n--- Milestone 4: Implementing Model 3 (Optional): Competitive Pricing Model ---")

# 1. Calculate geographic proximity of nearby parking spaces
# Get unique parking lot locations
parking_lots_geo = df[['SystemCodeNumber', 'Latitude', 'Longitude']].drop_duplicates().set_index('SystemCodeNumber')

# Haversine distance function (in km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371 # Earth radius in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Calculate distances between all unique parking lots
# This creates a distance matrix which is computationally intensive for large number of lots,
# but feasible for 14 lots.
lot_ids = parking_lots_geo.index
distances = pd.DataFrame(index=lot_ids, columns=lot_ids, dtype=float)

for i in range(len(lot_ids)):
    for j in range(i, len(lot_ids)):
        lot1 = lot_ids[i]
        lot2 = lot_ids[j]
        lat1, lon1 = parking_lots_geo.loc[lot1]['Latitude'], parking_lots_geo.loc[lot1]['Longitude']
        lat2, lon2 = parking_lots_geo.loc[lot2]['Latitude'], parking_lots_geo.loc[lot2]['Longitude']

        dist = haversine(lat1, lon1, lat2, lon2)
        distances.loc[lot1, lot2] = dist
        distances.loc[lot2, lot1] = dist # Symmetric matrix

# Fill diagonal with 0 (distance to self is 0)
np.fill_diagonal(distances.values, 0)

print("\nDistance matrix between parking lots calculated (in km):")
print(distances.head())


# 2. Determine competitor parking prices and factor them into pricing.
model3_pricing_results = []

# Initialize a dictionary to store the *latest calculated price* for each lot.
# This will act as the "current" price that competitors see.
# In a real-time system, this would be a state table managed by Pathway.
current_lot_prices_state = {lot_id: BASE_PRICE for lot_id in df['SystemCodeNumber'].unique()}

# Group the DataFrame by Timestamp to process all lots for a given time step
# This ensures that when calculating competitor prices for a given timestamp,
# we use the prices of competitors that were also calculated for that same timestamp.
for timestamp, time_group_df in df.groupby('Timestamp'):
    # Create a temporary dictionary to store prices calculated for this specific timestamp
    # This prevents using prices from *future* timestamps as competitor prices.
    prices_this_timestamp = {}

    for index, row in time_group_df.iterrows():
        lot_id = row['SystemCodeNumber']
        normalized_demand = row['NormalizedDemand'] # Use the normalized demand from Model 2

        # Get nearby competitors for the current lot based on the distance threshold
        nearby_competitors = distances.loc[lot_id][distances.loc[lot_id] <= COMPETITOR_DISTANCE_THRESHOLD_KM].index.tolist()
        if lot_id in nearby_competitors:
            nearby_competitors.remove(lot_id) # Remove self from competitors list

        # Get competitor prices from the 'current_lot_prices_state'
        # This simulates using the most recently available prices of competitors.
        competitor_prices = [current_lot_prices_state[comp] for comp in nearby_competitors if comp in current_lot_prices_state]

        # Calculate average competitor price (default to BASE_PRICE if no nearby competitors)
        avg_competitor_price = np.mean(competitor_prices) if competitor_prices else BASE_PRICE

        # Competitive Logic: Calculate competitive advantage factor
        # If avg_competitor_price is higher than my current base price, it's an advantage.
        # If it's lower, it's a disadvantage.
        competitive_advantage_factor = 0.0
        if avg_competitor_price > 0: # Avoid division by zero if prices are somehow 0
            # Calculate relative difference from my base price.
            # A positive value means competitors are generally more expensive.
            relative_price_diff = (avg_competitor_price - BASE_PRICE) / BASE_PRICE
            competitive_advantage_factor = ETA_COMPETITIVE * relative_price_diff

        # Model 3 Price Calculation: Combine Model 2's logic with competitive factor
        # Pricet = BasePrice · (1 + λ · NormalizedDemand + η · CompetitiveAdvantage)
        model3_price = BASE_PRICE * (1 + LAMBDA_MODEL2 * normalized_demand + competitive_advantage_factor)

        # Apply global price bounds
        model3_price = max(MIN_PRICE_GLOBAL, min(MAX_PRICE_GLOBAL, model3_price))

        # Store the calculated price for this lot for the *current* timestamp
        prices_this_timestamp[lot_id] = model3_price

        # Store results for this record
        model3_pricing_results.append({
            'Timestamp': timestamp,
            'SystemCodeNumber': lot_id,
            'NormalizedDemand': normalized_demand,
            'AvgCompetitorPrice': avg_competitor_price,
            'CompetitiveAdvantageFactor': competitive_advantage_factor,
            'Model3_Price': model3_price
        })

    # After processing all lots for the current timestamp, update the global state
    # with the newly calculated prices. This ensures the next timestamp group
    # uses the most up-to-date competitor prices.
    for lot_id, price in prices_this_timestamp.items():
        current_lot_prices_state[lot_id] = price


# Convert Model 3 results to a DataFrame
model3_df_results = pd.DataFrame(model3_pricing_results)
print("\nModel 3 simulation complete.")
print("Sample of Model 3 pricing results:")
print(model3_df_results.head())

# Merge Model 3 prices back to the main DataFrame
df = df.merge(model3_df_results[['Timestamp', 'SystemCodeNumber', 'Model3_Price', 'AvgCompetitorPrice', 'CompetitiveAdvantageFactor']],
              on=['Timestamp', 'SystemCodeNumber'],
              how='left')
print("\nModel 3 prices merged back to main DataFrame.")
print(df.head())


# --- Milestone 5: Real-Time Simulation with Pathway Integration (Conceptual Code) ---

print("\n\n--- Milestone 5: Conceptual Code for Pathway Real-Time Simulation ---")
print("This section outlines how Pathway would be used for real-time data ingestion and processing.")
print("Actual execution requires Pathway installation (`pip install pathway`) and a proper setup.")
print("The code below is illustrative and not directly executable without a Pathway environment.")

"""
import pathway as pw

# Define your pricing functions to be used within Pathway's map/reduce operations
# These functions would operate on Pathway Table rows.

# Constants (re-defined for clarity in Pathway context, but ideally passed or globally accessible)
# BASE_PRICE, MIN_PRICE_GLOBAL, MAX_PRICE_GLOBAL, ALPHA_MODEL1, etc. would be accessible.

def pathway_calculate_model1_price(row, current_price_state):
    # In Pathway, state management is explicit. `current_price_state` would be a pw.StateTable
    # or similar mechanism. This function would be part of a `pw.map` or `pw.reduce` operation.
    lot_id = row.SystemCodeNumber
    occupancy_rate = row.OccupancyRate
    prev_price = current_price_state.get(lot_id, BASE_PRICE) # Retrieve previous price from state
    next_price = prev_price + ALPHA_MODEL1 * occupancy_rate
    return max(MIN_PRICE_GLOBAL, min(MAX_PRICE_GLOBAL, next_price))

def pathway_calculate_demand(row):
    return (
        ALPHA_OCCUPANCY * row.OccupancyRate +
        ALPHA_QUEUELENGTH * row.QueueLength +
        ALPHA_TRAFFIC * row.TrafficConditionEncoded +
        ALPHA_SPECIALDAY * row.IsSpecialDay +
        ALPHA_VEHICLETYPE * row.VehicleTypeWeight
    )

def pathway_calculate_model2_price(row):
    # Assumes NormalizedDemand is already calculated on the row
    price = BASE_PRICE * (1 + LAMBDA_MODEL2 * row.NormalizedDemand)
    return max(MIN_PRICE_GLOBAL, min(MAX_PRICE_GLOBAL, price))

def pathway_calculate_model3_price(row, competitor_prices_table):
    # This is complex. `competitor_prices_table` would be another Pathway table
    # containing the latest prices of all lots. This function would involve a join
    # operation in Pathway to get competitor prices for the current lot.
    lot_id = row.SystemCodeNumber
    normalized_demand = row.NormalizedDemand

    # Conceptual: Get competitor prices from `competitor_prices_table` based on `distances` matrix
    # This would involve a complex join and aggregation in Pathway.
    # For illustration, let's assume `row.AvgCompetitorPrice` and `row.CompetitiveAdvantageFactor`
    # are already available from a prior Pathway join/computation.
    avg_competitor_price = row.AvgCompetitorPrice # From a joined table
    competitive_advantage_factor = row.CompetitiveAdvantageFactor # From a joined table

    model3_price = BASE_PRICE * (1 + LAMBDA_MODEL2 * normalized_demand + competitive_advantage_factor)
    return max(MIN_PRICE_GLOBAL, min(MAX_PRICE_GLOBAL, model3_price))


print("\nPathway setup involves:")
print("- Defining input data streams (e.g., from CSV, Kafka, custom generators).")
print("- Applying transformations (`.select`, `.map`, `.filter`) and stateful computations (`.reduce`, `pw.StateTable`).")
print("- Managing state for models that depend on previous values (e.g., Model 1, Model 3 for competitor prices).")
print("- Defining output sinks (e.g., print to console, write to another database, feed to a visualization tool).")
print("Refer to Pathway documentation for detailed API usage for stateful transformations, joins, and deployment.")


# --- Milestone 6: Visualization Implementation (Bokeh) ---

print("\n\n--- Milestone 6: Bokeh Visualization ---")
print("These plots demonstrate the pricing behavior using the historical data.")
print("In a real-time Pathway application, these Bokeh plots would be continuously updated")
print("by streaming data from the Pathway output table to a Bokeh server or directly within a dashboard.")

# 1. Real-time pricing line plots for each parking space
print("\n--- Real-time Pricing Line Plots for Each Parking Space (Sample Lots) ---")

# Select a few parking lots to visualize (e.g., first 5 unique lots)
sample_lot_ids = df['SystemCodeNumber'].unique()[:5]
plot_df_sample_lots = df[df['SystemCodeNumber'].isin(sample_lot_ids)].copy()

# Melt the DataFrame to easily plot multiple price models for each lot
plot_df_melted_models = plot_df_sample_lots.melt(
    id_vars=['Timestamp', 'SystemCodeNumber'],
    value_vars=['Model1_Price', 'Model2_Price', 'Model3_Price'],
    var_name='Model',
    value_name='Price'
)

# Create a ColumnDataSource for Bokeh (useful for potential live updates)
source_model_prices = ColumnDataSource(plot_df_melted_models)

# Create the plot figure
p_prices = figure(
    title="Dynamic Pricing Over Time (Sample Lots)",
    x_axis_label="Time",
    y_axis_label="Price ($)",
    x_axis_type="datetime",
    height=450,
    width=900,
    tools="pan,wheel_zoom,box_zoom,reset,save",
    sizing_mode="scale_width"
)

# Define colors for different lots (using Category10 palette)
unique_lots_in_plot = plot_df_melted_models['SystemCodeNumber'].unique().tolist()
colors_for_lots = Category10[len(unique_lots_in_plot)]
color_map_lots = {lot: colors_for_lots[i] for i, lot in enumerate(unique_lots_in_plot)}

# Add lines for each parking lot and model
for i, lot_id in enumerate(unique_lots_in_plot):
    lot_data = plot_df_melted_models[plot_df_melted_models['SystemCodeNumber'] == lot_id]
    for model_name in ['Model1_Price', 'Model2_Price', 'Model3_Price']:
        model_data = lot_data[lot_data['Model'] == model_name]
        line_dash_style = 'solid' # Default
        if 'Model1' in model_name:
            line_dash_style = 'solid'
        elif 'Model2' in model_name:
            line_dash_style = 'dashed'
        elif 'Model3' in model_name:
            line_dash_style = 'dotted'

        p_prices.line(
            x='Timestamp',
            y='Price',
            source=ColumnDataSource(model_data),
            legend_label=f"{lot_id} - {model_name.replace('_Price', '')}",
            color=color_map_lots[lot_id],
            line_dash=line_dash_style,
            line_width=2,
            alpha=0.7
        )

p_prices.xaxis.formatter = DatetimeTickFormatter(hours="%H:%M", days="%m/%d", months="%m/%Y")
p_prices.legend.location = "top_left"
p_prices.legend.click_policy = "hide" # Allow clicking legend to hide/show lines
p_prices.legend.label_text_font_size = "8pt" # Adjust font size for better readability if many legends

show(p_prices)


# 2. Comparison with competitor prices (for a single selected lot)
print("\n--- Price Comparison with Competitors (for a selected lot) ---")

# Choose a single lot to focus on for competitor comparison
focus_lot_id = df['SystemCodeNumber'].unique()[0] # Example: first unique lot

# Identify nearby competitors based on the distance matrix
nearby_competitors_for_focus = distances.loc[focus_lot_id][distances.loc[focus_lot_id] <= COMPETITOR_DISTANCE_THRESHOLD_KM].index.tolist()
if focus_lot_id in nearby_competitors_for_focus:
    nearby_competitors_for_focus.remove(focus_lot_id) # Remove self

# Limit to a few competitors for plot clarity if many exist
competitor_display_ids = nearby_competitors_for_focus[:3] # Show up to 3 closest competitors

if not competitor_display_ids:
    print(f"No nearby competitors found for {focus_lot_id} within {COMPETITOR_DISTANCE_THRESHOLD_KM}km for visualization.")
    print("Plotting only the focus lot's price.")
    all_display_lots_comp = [focus_lot_id]
else:
    print(f"Displaying prices for {focus_lot_id} and competitors: {competitor_display_ids}")
    all_display_lots_comp = [focus_lot_id] + competitor_display_ids


# Prepare data for competitor comparison plot (focus on Model 3 prices)
competitor_plot_df = df[df['SystemCodeNumber'].isin(all_display_lots_comp)].copy()

# Melt the DataFrame to plot prices for multiple lots
competitor_plot_df_melted = competitor_plot_df.melt(
    id_vars=['Timestamp', 'SystemCodeNumber'],
    value_vars=['Model3_Price'], # Focus on Model 3 price for competitive analysis
    var_name='Model',
    value_name='Price'
)


p_competitors = figure(
    title=f"Price Comparison for {focus_lot_id} vs. Competitors (Model 3)",
    x_axis_label="Time",
    y_axis_label="Price ($)",
    x_axis_type="datetime",
    height=450,
    width=900,
    tools="pan,wheel_zoom,box_zoom,reset,save",
    sizing_mode="scale_width"
)

# Define colors for the focus lot and its competitors
colors_for_comp_plot = Category10[max(3, len(all_display_lots_comp))] # Ensure at least 3 colors
comp_color_map = {lot: colors_for_comp_plot[i] for i, lot in enumerate(all_display_lots_comp)}

for lot_id in all_display_lots_comp:
    lot_data = competitor_plot_df_melted[competitor_plot_df_melted['SystemCodeNumber'] == lot_id]
    line_dash = 'solid' if lot_id == focus_lot_id else 'dashed' # Focus lot is solid, competitors are dashed
    p_competitors.line(
        x='Timestamp',
        y='Price',
        source=ColumnDataSource(lot_data),
        legend_label=f"{lot_id} Price",
        color=comp_color_map[lot_id],
        line_dash=line_dash,
        line_width=2,
        alpha=0.8
    )

p_competitors.xaxis.formatter = DatetimeTickFormatter(hours="%H:%M", days="%m/%d", months="%m/%Y")
p_competitors.legend.location = "top_left"
p_competitors.legend.click_policy = "hide"
p_competitors.legend.label_text_font_size = "8pt"

show(p_competitors)

# 3. Occupancy Rate vs. Price (Example for a single lot)
print("\n--- Occupancy Rate vs. Price (Example for a single lot) ---")

# Choose a single lot to visualize
occupancy_lot_id = df['SystemCodeNumber'].unique()[1] # Example: second unique lot
occupancy_plot_df = df[df['SystemCodeNumber'] == occupancy_lot_id].copy()

# Create a ColumnDataSource
source_occupancy_price = ColumnDataSource(occupancy_plot_df)

p_occupancy = figure(
    title=f"Occupancy Rate vs. Model 3 Price for {occupancy_lot_id}",
    x_axis_label="Occupancy Rate",
    y_axis_label="Model 3 Price ($)",
    height=400,
    width=700,
    tools="pan,wheel_zoom,box_zoom,reset,save",
    sizing_mode="scale_width"
)

# Plot scatter points (or a line if ordered by time)
p_occupancy.circle(
    x='OccupancyRate',
    y='Model3_Price',
    source=source_occupancy_price,
    size=5,
    alpha=0.6,
    color="blue"
)

show(p_occupancy)

print("\n--- All Visualizations Generated ---")
print("Remember to include these visualizations and their interpretations in your final report.")

--- Milestone 1: Data Understanding, Exploration, and Preprocessing ---
Loading the dataset 'dataset.csv'...
Dataset loaded successfully.

--- Initial Data Overview ---

First 5 rows of the dataset:
   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   
3                    low      

ValueError: time data "13-10-2016 07:57:00" doesn't match format "%m-%d-%Y %H:%M:%S", at position 162. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.